# 3 Ejemplo

In [1]:
from transformers import AutoModelForCausalLM

from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.models.transformers.transformers_model import TransformersModel, TransformersModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters

INFO 11-17 16:11:12 [__init__.py:216] Automatically detected platform cuda.


In [2]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
BENCHMARKS = "lighteval|gsm8k|1|1" # suite|task|few_shot|truncate_few_shots

In [3]:
evaluation_tracker = EvaluationTracker(output_dir="./results", save_details=True, push_to_hub=False)
pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.NONE,
    max_samples=2
)

model = AutoModelForCausalLM.from_pretrained(
  MODEL_NAME, device_map="auto"
)
config = TransformersModelConfig(model_name=MODEL_NAME, batch_size=1)
model = TransformersModel.from_model(model, config)

pipeline = Pipeline(
    model=model,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    tasks=BENCHMARKS,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
--max_samples WAS SET. THESE NUMBERS ARE ONLY PARTIAL AND SHOULD NOT BE USED FOR COMPARISON UNLESS YOU KNOW WHAT YOU ARE DOING.
If you want to use extended_tasks, make sure you installed their dependencies using `pip install -e .[extended_tasks]`.
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'gsm8k' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


In [4]:

results = pipeline.evaluate()
pipeline.show_results()
results = pipeline.get_results()

You cannot select the number of dataset splits for a generative evaluation at the moment. Automatically inferring.
Splits: 100%|██████████| 1/1 [02:04<00:00, 124.15s/it]

|      Task       |Version|     Metric     |Value|   |Stderr|
|-----------------|------:|----------------|----:|---|-----:|
|all              |       |extractive_match|  0.5|±  |   0.5|
|lighteval:gsm8k:1|      0|extractive_match|  0.5|±  |   0.5|



In [5]:
results

{'config_general': {'lighteval_sha': '?',
  'num_fewshot_seeds': 1,
  'max_samples': 2,
  'job_id': 0,
  'start_time': 14904.462102192,
  'end_time': 15040.258730416,
  'total_evaluation_time_secondes': '135.79662822400132',
  'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct',
  'model_sha': '8afb486c1db24fe5011ec46dfbe5b5dccdb575c2',
  'model_dtype': torch.float32,
  'model_size': '29.92 GB',
  'generation_parameters': {},
  'config': None},
 'results': {'lighteval:gsm8k:1': defaultdict(float,
              {'extractive_match': 0.5, 'extractive_match_stderr': 0.5}),
  'all': {'extractive_match': 0.5, 'extractive_match_stderr': 0.5}},
 'versions': {'lighteval:gsm8k:1': 0},
 'config_tasks': {'lighteval:gsm8k': LightevalTaskConfig(name='gsm8k', prompt_function=<function gsm8k at 0x78c79a3cb240>, hf_repo='gsm8k', hf_subset='main', metric=(SampleLevelMetric(metric_name='extractive_match', higher_is_better=True, category=<MetricCategory.GENERATIVE: '3'>, use_case=<MetricUseCase.ACCURACY: 


<p style="page-break-after:always;"></p>
